In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

In [2]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

데이터셋 로딩하기 (Cifar-10)

import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:75%]'],
    shuffle_files=True,
    with_info=True,
)

ds_train


ds_info.features

ds_train

In [ ]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [ ]:
# Tensorflow 데이터셋을 로드하면 꼭 feature 정보를 확인해 보세요. 
print(ds_info.features)

In [ ]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

In [ ]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [ ]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
ds_info.features["label"].num_classes

In [ ]:
ds_info.features["label"].names

In [ ]:
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
fig = tfds.show_examples(ds_test, ds_info)

In [3]:
def build_residual_block(input_layer,
                    num_cnn=2, 
                    channel=64,
                    block_num=1,
                    kernel_size=(3,3), chunk_num=0
                   ):
    
    # 입력 레이어
    x = input_layer
    
    # CNN 레이어
    for cnn_num in range(num_cnn):
        if chunk_num!=0 and block_num==0 and cnn_num==0:
            x = keras.layers.Conv2D(filters=channel,kernel_size=kernel_size,activation='relu',strides=2,
                                    kernel_initializer='he_normal',padding='same',name=f'{chunk_num}_block{block_num}_conv{cnn_num}')(x)
        else:
            x = keras.layers.Conv2D(filters=channel,kernel_size=kernel_size,activation='relu',
                                    kernel_initializer='he_normal',padding='same',name=f'{chunk_num}_block{block_num}_conv{cnn_num}')(x)
        x = keras.layers.BatchNormalization(axis=3)(x)
        if cnn_num < num_cnn-1:
            x = keras.layers.Activation('relu')(x)
        
    return x

resnet_input_layer = keras.layers.Input(shape=(224,224,3))   # 입력 레이어 생성
resnet_block_output = build_residual_block(resnet_input_layer)    # VGG 블록 생성

##### 블록 1개짜리 model 생성
model = keras.Model(inputs=resnet_input_layer, outputs=resnet_block_output)  

model.summary()

2-5. Ablation Study 실습 (3) VGG Complete Model

In [4]:
import copy

In [5]:
def build_resnet(input_shape=(224,224,3),
              num_block_list=[3, 4, 6, 3],
              channel_list=[64,128,256,512],
              num_classes=10, is_50=False): ## class for cats_n_dogs : 2
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    x = input_layer

    
    ############
    #CONV1
    x = keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=2, name=f'conv1')(x)
    x = keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, name=f'initial_maxpooling')(x)
    ############
    
    num_cnn = 2
    
    skip_connection= x

    for i, (block, channel) in enumerate(zip(num_block_list, channel_list)):
        for j in range(block):
            if j!=0:
                skip_connection=x
            x = build_residual_block(
                x,
                num_cnn=num_cnn, 
                channel=channel,
                block_num=j, chunk_num=i
            )

            print('x: ', x)
            print('s: ', skip_connection)

            print('chunk_num: {}, block_num: {}, add identity==================='.format(i, j))
            if i==0:
                x=tf.keras.layers.Add()([x, skip_connection])
        print('--------------------reduce in size')
        
#         skip_connection= tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)
#         if block_num!=1 and new_block==0:
#             x = keras.layers.Conv2D(filters=channel,kernel_size=kernel_size,activation='relu',
#                                     kernel_initializer='he_normal',padding='same', strides=2, name=f'block{block_num}_conv{cnn_num}')(x)
            
#         else:
        

# #         skip_connection= keras.layers.
        
    
#     output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(1000, activation='relu', name='fc1')(x)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [6]:
resnet_34 = build_resnet(input_shape=(224,224,3), is_50=False)


x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_1/FusedBatchNormV3:0', description="created by layer 'batch_normalization_1'")
s:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='initial_maxpooling/MaxPool:0', description="created by layer 'initial_maxpooling'")
chunk_num: 0, block_num: 0, add identity===================
x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_3/FusedBatchNormV3:0', description="created by layer 'batch_normalization_3'")
s:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='add/add:0', description="created by layer 'add'")
chunk_num: 0, block_num: 1, add identity===================
x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_5/FusedBatchNormV3:0', descriptio

In [231]:
resnet_34 = build_resnet(input_shape=(224,224,3), is_50=False)


x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_1795/FusedBatchNormV3:0', description="created by layer 'batch_normalization_1795'")
s:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='initial_maxpooling/MaxPool:0', description="created by layer 'initial_maxpooling'")
chunk_num: 0, block_num: 0, add identity===================
x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_1797/FusedBatchNormV3:0', description="created by layer 'batch_normalization_1797'")
s:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='add_78/add:0', description="created by layer 'add_78'")
chunk_num: 0, block_num: 1, add identity===================
x:  KerasTensor(type_spec=TensorSpec(shape=(None, 54, 54, 64), dtype=tf.float32, name=None), name='batch_normalization_1799/FusedBatch

In [229]:
resnet_34.summary()

Model: "model_63"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_82 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 109, 109, 64) 9472        input_82[0][0]                   
__________________________________________________________________________________________________
initial_maxpooling (MaxPooling2 (None, 54, 54, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
0_block0_conv0 (Conv2D)         (None, 54, 54, 64)   36928       initial_maxpooling[0][0]         
___________________________________________________________________________________________

2) VGG-19

In [216]:
# 원하는 블록의 설계에 따라 매개변수로 리스트를 전달해 줍니다.
vgg_19 = build_vgg(
    num_cnn_list=[2,2,4,4,4],
    channel_list=[64,128,256,512,512]
)

vgg_19.summary()

NameError: name 'build_vgg' is not defined

2-6. Ablation Study 실습 (4) VGG-16 vs VGG-19


In [ ]:
BATCH_SIZE = 256
EPOCH = 50

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [ ]:
vgg_16.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_16 = vgg_16.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
vgg_19.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_19 = vgg_19.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_16.history['loss'], 'r')
plt.plot(history_19.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_16.history['val_accuracy'], 'r')
plt.plot(history_19.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()

2-7. 프로젝트: ResNet Ablation Study
1) ResNet 기본 블록 구성하기
이제 실전으로 돌아와서 ResNet-34와 ResNet-50 네트워크를 직접 만든다고 생각해 봅시다. 우선 ResNet-34와 ResNet-50에서 사용되는 블록의 공통점을 찾고 차이점을 찾아봅시다.

Deep Residual Learning for Image Recognition(paper)
빛과 같은 카이밍허님의 깃헙
content img
Q1. ResNet논문에 소개된 위 표에서, ResNet 34와 50에서 사용된 블록들의 공통점과 차이점을 비교해보고 그 특징을 정리해봅시다.

.
예시답안
예시답안
ResNet-34와 ResNet-50은 모두 conv block이라고 불리는 블록 구조를 각각 3, 4, 6, 3개씩 반복해서 쌓은 형태라는 공통점을 가지고 있습니다. 그러나 ResNet-34와 ResNet-50의 블록 내부 구성은 약간 다릅니다. ResNet-34의 경우 Block은 3x3 kernel인 Convolution layer로만 구성되어있지만, ResNet-50은 1x1 Convolution이 앞뒤로 붙어 더 많은 레이어를 한 블록 내에 가지게 됩니다.

위의 퀴즈의 답을 어느 정도 스스로 찾아내셨다면 ResNet의 기본 블록, 또는 Residual 블록(residual block)이 어떻게 구성되어야 할지 감이 오셨을 것입니다. Learning by Doing 이라고 하죠! 이번에는 ResNet의 블록을 직접 만들어 봅시다! 혹시나 모델 내의 앞과 뒤에서 반복되지 않을 것 같은 부분은 함수로 구현하지 않아도 됩니다.

우리가 지금 만들어야 하는 것은 블록을 생성하는 함수입니다. 처음이라 너무 막연하실 수 있겠지만 여러분들은 이미 몇 번의 경험이 있습니다.
일단은 VGG에서도 블록을 생성하는 함수를 만들었습니다. ResNet이라 하더라도 아이디어의 기본 원리는 다르지 않습니다.

그리고, 여러분들은 아마 이전에 VGG-16이나 ResNet-50 모델을 만들어서 훈련까지 시켜본 경험들이 있으실 것입니다. 코드를 스스로 구현한 것이 아니라 공개된 오픈소스를 활용해서 빠르게 구성했겠지만, 블록 생성 함수가 어떤 형태로 만들어지는지를 이미 경험해 보신 바 있을 테니 그것을 참고하셔도 도움이 될 것입니다.

하지만 이번 단계에서는 가급적 ResNet 논문을 통해서 구현 방법에 대한 정보를 얻으시기를 권합니다. 논문만 보고 스스로 구현해 보는 경험을 통해 딥러닝 개발자로서의 내공과 자신감이 다져지게 될 것입니다.

2) ResNet-34, ResNet-50 Complete Model
ResNet 모델 구현 시 Sequential API나 Subclass API를 사용한다면, 그 과정에서 모델 단위로 기존의 코드를 재활용했을 때 model.summary() 호출 시 서브모델 내부의 레이어 구성이 생략되고 서브모델 단위로만 출력될 우려가 있습니다. 모델 구성만을 위해서는 그런 방법도 무방하지만, 가급적 이번 실습에서는 VGG 실습 예시에서처럼 Functional API 를 구성하는 방식을 사용할 것을 권합니다.

ResNet-34
VGG와 같이 블록을 만드는 함수를 사용해서 직접 전체 모델을 만들어 봅시다. ResNet-34와 ResNet-50의 차이에 따라 달라지는 구성(configuration)을 함수에 전달해서 같은 생성 함수 build_resnet()를 통해서 ResNet의 여러 가지 버전들을 모두 만들어 낼 수 있도록 해야 합니다.

다음의 코드를 실행하면,

In [ ]:
resnet_50 = build_resnet(input_shape=(32, 32,3), is_50=True)
resnet_50.summary()

이러한 결과와 '비슷'하게 나와야 합니다. 참고하세요!
ResNet-50은 구현마다 약간씩 다르게 나올 수 있습니다. 왜냐하면 Residual 블록의 Batch Normalization과 ReLU 레이어의 위치가 논문에 명시되지 않았기 때문이죠!

하지만 ResNet을 고안한 4인방은 이에 대해 또 한번 논문을 씁니다. Residual 블록에 대해서 자세하게 씁니다. 이 논문을 읽어보면 오리지널 ResNet-50에 대한 힌트가 약.간. 있을 수도 있습니다~!

Identity Mappings in Deep Residual Networks

3) 일반 네트워크(plain network) 만들기
블록 코드 수정하기
우리는 앞에서 ResNet 모델을 구현했습니다. ResNet의 핵심 아이디어는 skip connection과 residual network기 때문에, ResNet의 효과를 보여주기 위해서는 skip connection이 없는 일반 네트워크(plain net)가 필요합니다. 위에서 ResNet 블록을 만들기 위한 함수를 그대로 활용해서 skip connection이 없는 블록을 만들 수 있도록 기능을 추가해 주세요!

전체 함수 코드 수정하기
이제 위에서 만든 블록 함수를 토대로 전체 네트워크를 만들 수 있도록 전체 네트워크 코드를 수정합시다. ResNet-50과 ResNet-34, 그리고 같은 레이어를 가지지만 skip connection이 없는 PlainNet-50과 PlainNet-34를 만들 수 있는 함수 build_plainnet()를 만들어 보세요. 이때 입력 이미지의 크기는 (224, 224, 3)으로 해주세요.

4) ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34
Ablation Study
이제 VGG-16, 19 예제와 같이 ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34에 대해서 학습을 진행해 봅니다. 그리고 결과를 비교해 봅시다! ResNet은 많은 레이어와 Pooling을 거치므로 CIFAR-10에서는 오버피팅(overfitting)으로 잘 동작하지 않을 수 있습니다. 레이어가 많고 학습해야 할 변수(parameter)가 많은 데 비해, 데이터 수가 많지 않기 때문이지요. 224x224 픽셀 크기의 데이터셋을 찾아서 실험해 보도록 합시다. 학습은 끝까지 시키기엔 시간이 없으니 확인을 위한 정도의 epoch로 설정해 주세요.

어떤 데이터셋을 사용하셔도 무방하지만, 얼른 떠오르는 것이 없다면 tensorflow-datasets에서 제공하는 cats_vs_dogs 데이터셋을 추천합니다. 아마 이 데이터셋도 다루어 보신 적이 있을 것입니다. Tensorflow에서 제공하는 데이터셋이므로 오늘 VGG 학습에 사용했던 CIFAR-10을 로딩하는 것과 같은 방법으로 활용하실 수 있습니다.

시각화
학습을 통해 만들어진 history를 matplotlib로 시각화하여 두 모델의 검증 정확도(validation accuracy)와 검증 손실(validation loss)이 어떻게 다른지 살펴봅시다.

추가 실험
시간이 남으면 아래 카탈로그를 보고, tensorflow-datasets의 지원하는 다른 데이터셋에 실험을 해봅시다.

Tensorflow Datasets

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항	상세기준
1. ResNet-34, ResNet-50 모델 구현이 정상적으로 진행되었는가?

블록함수 구현이 제대로 진행되었으며 구현한 모델의 summary가 예상된 형태로 출력되었다.
2. 구현한 ResNet 모델을 활용하여 Image Classification 모델 훈련이 가능한가?

cats_vs_dogs 데이터셋으로 학습시 몇 epoch동안 안정적으로 loss 감소가 진행 확인되었다.
3. Ablation Study 결과가 바른 포맷으로 제출되었는가?

ResNet-34, ResNet-50 각각 plain모델과 residual모델을 동일한 epoch만큼 학습시켰을 때의 validation accuracy 기준으로 Ablation Study 결과표가 작성되었다.
프로젝트 업로드 (URL)

GitHub URL을 입력하신 후 하단의 [성취하기] 버튼을 눌러주세요.